In [ ]:
# pip install beautifulsoup4
# pip install requests
# pip install yfinance
# pip install datetime

In [ ]:
import bs4 as bs
import requests
import yfinance as yf
import datetime


/usr/local/lib/python3.10/dist-packages/yfinance/base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()


In [ ]:
resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
soup = bs.BeautifulSoup(resp.text, 'lxml')
table = soup.find('table', {'class': 'wikitable sortable'})

In [ ]:
tickers = []

for row in table.findAll('tr')[1:]:
    ticker = row.findAll('td')[0].text
    tickers.append(ticker)

In [ ]:
tickers = [s.replace('\n', '') for s in tickers]
print (tickers)

['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A', 'APD', 'ABNB', 'AKAM', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON', 'APA', 'AAPL', 'AMAT', 'APTV', 'ACGL', 'ADM', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'AXON', 'BKR', 'BALL', 'BAC', 'BK', 'BBWI', 'BAX', 'BDX', 'BRK.B', 'BBY', 'BIO', 'TECH', 'BIIB', 'BLK', 'BX', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF.B', 'BLDR', 'BG', 'CDNS', 'CZR', 'CPT', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'COR', 'CNC', 'CNP', 'CF', 'CHRW', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'COP', 'ED', 'STZ', 'CEG', 'COO', 'CPRT', 'GLW', 'CTVA', 'CSGP', 'COST', 'CTRA', 'CCI', 'CSX', 'CMI', 'CVS', 'DH

In [ ]:
# start = datetime.datetime(2024, 1, 1)
# end = datetime.datetime(2024, 1, 1)
# data = yf.download(tickers, start=start, end=end)

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Step 1: Scrape S&P 500 company list with ranks
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

table = soup.find("table", {"id": "constituents"})
rows = table.find_all("tr")[1:]  # excluding header row

data = []
for row in rows:
    cols = row.find_all("td")
    # rank = int(cols[0].text.strip())  # Rank is in the first column
    name = cols[1].text.strip()       # Company name is in the second column
    stock_symbol = cols[0].text.strip()
    website = ""  # No direct website available in this table
    investor_relation_website = ""  # No direct IR website available in this table
    sec_10k = ""  # You may need to find a reliable source to fetch this information
    market_cap = cols[3].text.strip()  # Market cap is in the fourth column

    data.append([
                  # rank,
                 name, stock_symbol, website, investor_relation_website, sec_10k, market_cap])

# Step 3: Organize data into DataFrame
columns = [
    # "Rank",
           "company_name", "stock_symbol", "website", "investor_relation_website", "SEC_10K", "market_cap"]
df = pd.DataFrame(data, columns=columns)

# Step 4: Export DataFrame to Jupyter Notebook
# df.to_csv("S&P_500_Companies.csv", index=False)


In [ ]:
%%time

for index, row in df.iterrows():
    symbol = row['stock_symbol']
    ticker = yf.Ticker(symbol)
    market_cap = ticker.info.get('marketCap', "N/A")
    df.at[index, 'market_cap'] = market_cap



CPU times: user 4.47 s, sys: 238 ms, total: 4.7 s
Wall time: 36.3 s


In [ ]:
%%time

def get_company_website(symbol):
    try:
        ticker = yf.Ticker(symbol)
        info = ticker.info
        website = info.get('website', "N/A")
        return website
    except Exception as e:
        print(f"Error fetching website for {symbol}: {e}")
        return "N/A"

# Iterate through each row in the DataFrame
for index, row in df.iterrows():
    symbol = row['stock_symbol']
    website = get_company_website(symbol)
    df.at[index, 'website'] = website



CPU times: user 4.13 s, sys: 203 ms, total: 4.34 s
Wall time: 23 s


In [ ]:
df.head()

,company_name,stock_symbol,website,investor_relation_website,SEC_10K,market_cap
0,3M,MMM,https://www.3m.com,,,50433875968
1,A. O. Smith,AOS,https://www.aosmith.com,,,11788848128
2,Abbott,ABT,https://www.abbott.com,,,197928206336
3,AbbVie,ABBV,https://www.abbvie.com,,,313365692416
4,Accenture,ACN,https://www.accenture.com,,,231661010944


In [ ]:
df[df['investor_relation_website'].notnull()]

,company_name,stock_symbol,website,investor_relation_website,SEC_10K,market_cap,latest_10k_link
0,3M,MMM,https://www.3m.com,,,50433875968,N/A
1,A. O. Smith,AOS,https://www.aosmith.com,,,11788848128,N/A
2,Abbott,ABT,https://www.abbott.com,,,197928206336,N/A
3,AbbVie,ABBV,https://www.abbvie.com,,,313365692416,N/A
4,Accenture,ACN,https://www.accenture.com,,,231661010944,N/A
...,...,...,...,...,...,...,...
498,Yum! Brands,YUM,https://www.yum.com,,,37808549888,N/A
499,Zebra Technologies,ZBRA,https://www.zebra.com,,,14153520128,N/A
500,Zimmer Biomet,ZBH,https://www.zimmerbiomet.com,,,26062018560,N/A
501,Zions Bancorporation,ZION,https://www.zionsbancorporation.com,,,6020937728,N/A


In [ ]:
df.to_csv('20240219_company_list.csv',index=False)

In [ ]:
# !pip install gspread oauth2client df2gspread
